Проверка выиигрыша моделей ставок на статистике испании

In [6]:
import pandas as pd
import numpy as np
import random

In [139]:
data=pd.read_csv('SP1.csv')
data=data[['Date','HomeTeam','AwayTeam','B365H','B365D','B365A','FTR']]
data['B365H_p']=round(1/data['B365H'],2)
data['B365D_p']=round(1/data['B365D'],2)
data['B365A_p']=round(1/data['B365A'],2)
data_len=data.shape[0]
data.head(15)

,Date,HomeTeam,AwayTeam,B365H,B365D,B365A,FTR,B365H_p,B365D_p,B365A_p
0,19/08/16,La Coruna,Eibar,1.95,3.40,4.20,H,0.51,0.29,0.24
1,19/08/16,Malaga,Osasuna,1.62,3.75,6.00,D,0.62,0.27,0.17
2,20/08/16,Barcelona,Betis,1.08,11.00,29.00,H,0.93,0.09,0.03
3,20/08/16,Granada,Villarreal,2.50,3.20,3.00,D,0.40,0.31,0.33
4,20/08/16,Sevilla,Espanol,1.67,4.00,5.00,H,0.60,0.25,0.20
5,21/08/16,Ath Madrid,Alaves,1.20,6.50,17.00,D,0.83,0.15,0.06
6,21/08/16,Sociedad,Real Madrid,4.75,3.80,1.75,A,0.21,0.26,0.57
7,21/08/16,Sp Gijon,Ath Bilbao,4.00,3.50,1.95,H,0.25,0.29,0.51
8,22/08/16,Celta,Leganes,1.65,3.75,5.75,A,0.61,0.27,0.17
9,22/08/16,Valencia,Las Palmas,1.73,3.75,5.00,A,0.58,0.27,0.20


In [13]:
# Случайная модель - случайно ставим на победу одной из команд
def gen_rnd_model(l):
    model=list()
    result=['H','D','A']
    for i in range(l):
        model.append(random.choice(result))
    return model

In [14]:
# Проверяем модель. Возвращаем выыиигрышь и точность
def test_model (model,data,bet):
    guess_data=data[data.FTR==model]
    win=guess_data[(guess_data.FTR=='H')]['B365H'].sum()+guess_data[guess_data.FTR=='A']['B365A'].sum()+guess_data[guess_data.FTR=='D']['B365D'].sum()
    acc=round(guess_data.shape[0]/data.shape[0]*100,0)
    return round((win-data.shape[0])*bet,0), acc

In [34]:
def test_rnd_mean(data,bet):
    sum=list()
    acc=list()
    l=data.shape[0]
    for i in range(10000):
        model=gen_rnd_model(l)
        res=test_model(model,data,bet)
        sum.append(res[0])
        acc.append(res[1])
    return round(np.mean(sum),0),round(np.mean(acc),0)
test_rnd_mean(data,10)

(-115.0, 33.0)

In [17]:
# Ставим на победу одной стороны
def gen_side_model(l,res):
    model=list()
    for i in range(l):
        model.append(res)
    return model

H_model=gen_side_model(data_len,'H')
A_model=gen_side_model(data_len,'A')
D_model=gen_side_model(data_len,'D')
print ("H=",test_model(H_model,data,10))
print ("A=",test_model(A_model,data,10))
print ("D=",test_model(D_model,data,10))

H= (-195.0, 46.0)
A= (-292.0, 27.0)
D= (153.0, 27.0)


In [35]:
# Ставим зная результат
def gen_oracle_model(data):
    return data.FTR
ORACLE_model=gen_oracle_model(data)
print ("ORACLE=",test_model(ORACLE_model,data,10))

ORACLE= (4005.0, 100.0)


In [98]:
# p -вероятность в долях, l -длина
def get_choice(p,l):
    right=int(l*p)
    no=l-right
    items=[True]*right+[False]*no
    random.shuffle(items)
    return np.asarray(items)
    
def gen_oracle_prob_model(data,p):
    model=list()
    for d in data.FTR:
        if get_choice(p):
            model.append(d)
        else:
            model.append('L')
    return model

def gen_oracle_prob_model2(data,p):
    l=data.shape[0]
    c=get_choice(p,l)
    return data.FTR.where(c)

def test_oracle_prob_mean(data,bet,p):
    sum=list()
    acc=list()
    l=data.shape[0]
    for i in range(100):
        model=gen_oracle_prob_model2(data,p)
        res=test_model(model,data,bet)
        sum.append(res[0])
        acc.append(res[1])
    return round(np.mean(sum),0),round(np.mean(acc),0)


In [118]:
for i in range(10,110,10):
    prob = i/100
    print (test_oracle_prob_mean(data,10,prob))

(-1594.0, 10.0)
(-933.0, 20.0)
(-309.0, 30.0)
(258.0, 40.0)
(950.0, 49.0)
(1530.0, 59.0)
(2125.0, 69.0)
(2779.0, 79.0)
(3378.0, 89.0)
(4005.0, 100.0)


In [206]:
# Считаем количество очков для команды на начало матча
def c_team_points(team,match):
    pts=0
    past_matches_win_home=data[(data.index<match)&(data.HomeTeam==team)&(data.FTR=='H')]
    past_matches_win_away=data[(data.index<match)&(data.AwayTeam==team)&(data.FTR=='A')]
    past_matches_draw=data[(data.index<match)&((data.HomeTeam==team)|(data.AwayTeam==team))&(data.FTR=='D')]
    pts=past_matches_draw.shape[0]+3*(past_matches_win_home.shape[0]+past_matches_win_away.shape[0])
    return pts
# Расчитываем очки каждой команды на начало тура
def set_team_points(data):
    for match in range(data.shape[0]):
        ht=data.ix[match,'HomeTeam']
        at=data.ix[match,'AwayTeam']    
        hpts=c_team_points(ht,match)
        apts=c_team_points(at,match)
        data.ix[match,'HT_pts']=hpts
        data.ix[match,'AT_pts']=apts
set_team_points(data)
data.head(100)

,Date,HomeTeam,AwayTeam,B365H,B365D,B365A,FTR,B365H_p,B365D_p,B365A_p,HT_pts,AT_pts
0,19/08/16,La Coruna,Eibar,1.95,3.40,4.20,H,0.51,0.29,0.24,0,0
1,19/08/16,Malaga,Osasuna,1.62,3.75,6.00,D,0.62,0.27,0.17,0,0
2,20/08/16,Barcelona,Betis,1.08,11.00,29.00,H,0.93,0.09,0.03,0,0
3,20/08/16,Granada,Villarreal,2.50,3.20,3.00,D,0.40,0.31,0.33,0,0
4,20/08/16,Sevilla,Espanol,1.67,4.00,5.00,H,0.60,0.25,0.20,0,0
5,21/08/16,Ath Madrid,Alaves,1.20,6.50,17.00,D,0.83,0.15,0.06,0,0
6,21/08/16,Sociedad,Real Madrid,4.75,3.80,1.75,A,0.21,0.26,0.57,0,0
7,21/08/16,Sp Gijon,Ath Bilbao,4.00,3.50,1.95,H,0.25,0.29,0.51,0,0
8,22/08/16,Celta,Leganes,1.65,3.75,5.75,A,0.61,0.27,0.17,0,0
9,22/08/16,Valencia,Las Palmas,1.73,3.75,5.00,A,0.58,0.27,0.20,0,0


In [176]:
# На лидера
data.head(10)
def gen_lider_model(data):
    model=list()
    for match in range(data.shape[0]):
        if (data.ix[match,'HT_pts']==data.ix[match,'AT_pts']):
            res='D'
        elif (data.ix[match,'HT_pts']>data.ix[match,'AT_pts']):
            res='H'
        else:
            res='A'
        model.append(res)
    return model
lider_model=gen_lider_model(data)
print ("Lider=",test_model(lider_model,data,10))

Lider= (-205.0, 47.0)


In [216]:
# На форму (среднее за N последних матчей)

# Считаем форму для команды за последние last матчей на начало матча 
def c_team_res(team,match,last):
    past_matches=data[(data.index<match)&((data.HomeTeam==team)|(data.AwayTeam==team))].tail(last)
    past_matches_win_home=past_matches[(past_matches.HomeTeam==team)&(past_matches.FTR=='H')]
    past_matches_win_away=past_matches[(past_matches.AwayTeam==team)&(past_matches.FTR=='A')]
    past_matches_lose_home=past_matches[(past_matches.HomeTeam==team)&(past_matches.FTR=='A')]
    past_matches_lose_away=past_matches[(past_matches.AwayTeam==team)&(past_matches.FTR=='H')]
    pts=past_matches_win_home.shape[0]+past_matches_win_away.shape[0]-past_matches_lose_home.shape[0]-past_matches_lose_away.shape[0]
    return pts

# Расчитываем форму каждой команды на начало тура
def set_team_form(data,last):
    data_form=data
    for match in range(data_form.shape[0]):
        ht=data_form.ix[match,'HomeTeam']
        at=data_form.ix[match,'AwayTeam']    
        hpts=c_team_res(ht,match,last)
        apts=c_team_res(at,match,last)
        data_form.ix[match,'HT_form']=hpts
        data_form.ix[match,'AT_form']=apts
    return data_form

In [220]:
def gen_form_model(data,last):
    model=list()
    data_form=set_team_form(data,last)
    for match in range(data_form.shape[0]):
        if (data_form.ix[match,'HT_form']==data_form.ix[match,'AT_form']):
            res='D'
        elif (data_form.ix[match,'HT_form']>data_form.ix[match,'AT_form']):
            res='H'
        else:
            res='A'
        model.append(res)
    return model

for last in range(10):
    form_model=gen_form_model(data,last+1)
    print ("last=",last+1,"res",test_model(form_model,data,10))

last= 1 res (-335.0, 31.0)
last= 2 res (-62.0, 38.0)
last= 3 res (-231.0, 41.0)
last= 4 res (-400.0, 41.0)
last= 5 res (-351.0, 41.0)
last= 6 res (-396.0, 41.0)
last= 7 res (-376.0, 42.0)
last= 8 res (-194.0, 45.0)
last= 9 res (-264.0, 44.0)
last= 10 res (-246.0, 45.0)


### Выигрыш моделей при ставке 10$
    Случайная            = -115 , 33
    Всегда на хозяев     = -195 , 46
    Всегда на гостей     = -292 , 27
    Всегда на ничью      =  153 , 27
    Зная предсказания    =  4005.0, 100.0
    Пред. с вероятностью = -1594.0, 10.0)
                           -933.0, 20.0)
                           -309.0, 30.0)
                            258.0, 40.0)
                            950.0, 49.0)
                            1530.0, 59.0)
                            2125.0, 69.0)
                            2779.0, 79.0)
                            3378.0, 89.0)
                            4005.0, 100.0)
    На лидера          =   -205.0, 47.0
    На результат прошлой игры 
                         =last= 1 res (-335.0, 31.0)
                          last= 2 res (-62.0, 38.0)
                          last= 3 res (-231.0, 41.0)
                          last= 4 res (-400.0, 41.0)
                          last= 5 res (-351.0, 41.0)
                          last= 6 res (-396.0, 41.0)
                          last= 7 res (-376.0, 42.0)
                          last= 8 res (-194.0, 45.0)
                          last= 9 res (-264.0, 44.0)
                          last= 10 res (-246.0, 45.0)

In [ ]:


data.shape[0]